# **Análisis de Horas Extras a Partir de Sistemas Biométricos de Control de Asistencia**
### Proyecto Final Samsung Innovation Campus – Módulo de Python (EC04)

**Integrantes:**
- Ulices Chingo 
- Sofia Feijóo
- Brayan Maisincho
- Esteban Quiña
- Alan Palma

In [21]:
# Importar Librerías
import pandas as pd
import datetime
from datetime import datetime, timedelta
import os

In [2]:
def cargar_datos(path):
    try: 
        # Tomar el tipo de archivo
        tipo_archivo = path.split(".")[-1]
        print(f"Cargando archivo de tipo: .{tipo_archivo}")

        # Cargar el archivo según su tipo
        if tipo_archivo == "xls":
            data = pd.read_excel(path)
        else:
            data = pd.read_csv(path, delimiter=",")
            
        print("Archivo cargado exitosamente!")
        return data
    except:
        print("No se pudo acceder al archivo. Verificar que el archivo sea de tipo .csv, y que el archivo se encuetra en la dirección proporcionada.")
        return None

In [8]:
def procesamiento(df, col_name = None):

    # Rename columns
    if col_name is None:
        col_name = {'TiemInicio': 'Inicio', 'TiemFinal': 'Final', 'Fecha': 'Diferencia', '????': 'Date'}

    df.rename(columns=col_name, inplace=True)
    print("Columnas renombradas exitosamente.")

    # Transform the time in minutes

    Diff_min = []
    for i in df["Diferencia"]:
        (h, m) = i.split(':')
        min1 = int(h) * 60
        min2 = int(m)
        min = min1 + min2
        Diff_min.append(min)
    df["Diferencia en minutos"] = Diff_min

    #Tabla de  tiempo trabajado por empleado por día
    tiemposDia = []
    personas = []
    dias_trabajados = []

    i=0

    while i < (len(df["Nombre"])-1):
        cont=0
        tiempo_dia = df.loc[i, "Diferencia en minutos"]
        personas.append(df.loc[i, "Nombre"])
        dias_trabajados.append(df.loc[i, "Date"])
        while (df.loc[i, "Date"] == df.loc[i+cont+1, "Date"]) and (df.loc[i, "Nombre"] == df.loc[i+cont+1, "Nombre"]):
            tiempo_dia += df.loc[i+cont+1, "Diferencia en minutos"]
            cont += 1
        tiemposDia.append(tiempo_dia)
        i += cont+1

    tiempo_dia = df.loc[i, "Diferencia en minutos"]
    tiemposDia.append(tiempo_dia)
    personas.append(df.loc[i, "Nombre"])
    dias_trabajados.append(df.loc[i, "Date"])
    totDATA = pd.DataFrame({"Nombre": personas, "Minutos totales por dia": tiemposDia, "Fecha": dias_trabajados})

    return totDATA

In [23]:
import pandas as pd
from datetime import datetime

def horas_extras(totDATA, mes=None, feriados_csv=None):
    """
    """

    # Leer feriados desde CSV si se proporciona
    if feriados_csv:
        try:
            df_feriados = pd.read_csv(feriados_csv)
            if "Fecha" not in df_feriados.columns:
                raise ValueError("El archivo CSV debe contener una columna llamada 'Fecha'.")
            holydays = df_feriados["Fecha"].astype(str).tolist()
            print(f"{len(holydays)} feriados cargados desde {feriados_csv}")
        except Exception as e:
            print(f"Error al leer {feriados_csv}: {e}")
            holydays = []
    else:
        # Si no se proporciona CSV, preguntar manualmente
        holydays = []
        is_holiday = input(f"¿Hay feriados en este el mes de {mes}? (S/N): ").strip().upper()
        if is_holiday == "S":
            how_many = int(input("Ingrese el número de feriados: "))
            for i in range(how_many):
                holyi = input(f"Ingrese la fecha del feriado {i+1} (DD/MM/YYYY): ").strip()
                holydays.append(holyi)
        else:
            print("No se registraron feriados manuales.")

    # Identificar domingos y feriados (Horas 100%)
    sunday_holy = []
    for i in range(len(totDATA["Fecha"])):
        date_string = str(totDATA.loc[i, "Fecha"])
        try:
            date_object = datetime.strptime(date_string, "%d/%m/%Y")
        except ValueError:
            raise ValueError(f"Formato de fecha inválido: {date_string}. Use DD/MM/YYYY.")

        if date_object.weekday() == 6 or date_string in holydays:
            sunday_holy.append(1)
        else:
            sunday_holy.append(0)

    # Identificar sábados (Horas 50%)
    saturdays = []
    for i in range(len(totDATA["Fecha"])):
        date_string = str(totDATA.loc[i, "Fecha"])
        date_object = datetime.strptime(date_string, "%d/%m/%Y")
        saturdays.append(1 if date_object.weekday() == 5 else 0)

    # Añadir columnas al DataFrame
    totDATA["Sábado"] = saturdays
    totDATA["Domingo/Feriado"] = sunday_holy

    print("Cálculo de horas extras completado.")

    #Minutos extra 50% Y 100% por dia y por trabajador
    min50 = []
    min100 = []
    for i in range(len(totDATA["Fecha"])):
        #Dia normal de lunes a viernes
        if (totDATA.loc[i, "Sábado"] == 0) and (totDATA.loc[i, "Domingo/Feriado"] == 0):
            extra = totDATA.loc[i, "Minutos totales por dia"] - 480
            min50.append(extra)
            min100.append(int(0))
        #Sábado
        elif totDATA.loc[i, "Sábado"] == 1:
            extra = totDATA.loc[i, "Minutos totales por dia"]
            min50.append(extra)
            min100.append(int(0))
        #Dia especial (Domingo o feriado)
        elif totDATA.loc[i, "Domingo/Feriado"]:
            extra = totDATA.loc[i, "Minutos totales por dia"]
            min100.append(int(extra))
            min50.append(int(0))

    totDATA["Minutos extra 50%"] = min50
    totDATA["Minutos extra 100%"] = min100

In [24]:
def val_anormales(totDATA):
    """
    """
    #Identificar valores anormales
    for i in range(len(totDATA["Minutos extra 50%"])):
        if totDATA.loc[i, "Minutos extra 50%"] < int(0):
            print("No se cumplió la con la jornada completa en el siguiente día: ")
            display(totDATA.loc[i])

In [25]:
def generar_reporte(totDATA, file_name = "reporte_horas_extras.csv"):
    """
    """
    #Sumar minutos 50% y 100% por separado
    #Contar dias asistidos por trabajador
    listado = totDATA["Nombre"].unique()
    tot50 = [0 for k in range(len(listado))]
    tot100 = [0 for k in range(len(listado))]
    dias = [0 for k in range(len(listado))]
    for i in range(len(totDATA["Nombre"])):
        for j in range(len(listado)):
            if totDATA.loc[i, "Nombre"] == listado[j]:
                tot50[j] += totDATA.loc[i, "Minutos extra 50%"]
                tot100[j] += totDATA.loc[i, "Minutos extra 100%"]
                dias[j] +=1
    reporte = pd.DataFrame({"Nombre": listado, "Total minutos extra 50%": tot50, "Total minutos extra 100%": tot100})
    #display(reporte)

    #Tranformar a horas (decimal format) y añadir días asistidos
    hor50 = []
    hor100 = []
    for i in range(len(reporte["Nombre"])):
        h5 = reporte.loc[i, "Total minutos extra 50%"] / int(60)
        hor50.append(h5)
        h1 = reporte.loc[i, "Total minutos extra 100%"] / int(60)
        hor100.append(h1)
    reporte_final = pd.DataFrame({"Nombre": listado, "Total horas extras 50%": hor50, "Total horas extras 100%": hor100, "Dias asistidos": dias})
    reporte_final.sort_values(by='Total horas extras 50%')

    # Create un directorio en donde se guardará el reporte
    dir_output = "output_data"
    if os.path.isdir(dir_output):
        print(f"Directorio '{dir_output}' ya existe.")
    else:
        print(f"Directorio '{dir_output}' ha sido creado.")
        os.mkdir(dir_output)
    # Gurdar el reporte
    try:
        reporte_final.to_csv(dir_output + "/" + file_name, index=False)
        print(f"Reporte guardado exitosamente en {dir_output + '/' + file_name}")
    except Exception as e:
        print(f"Error al guardar el reporte: {e}")
    

In [26]:
path = "data/miscelaneosjunio2024.csv"

data = cargar_datos(path)

display(data)

Cargando archivo de tipo: .csv
Archivo cargado exitosamente!


,Nombre,TiemInicio,TiemFinal,Fecha,????
0,Manuel F,04:43,09:45,5:1,01/06/2024
1,Manuel F,06:55,12:38,5:43,03/06/2024
2,Manuel F,13:27,16:12,2:44,03/06/2024
3,Manuel F,06:56,12:40,5:43,04/06/2024
4,Manuel F,13:27,16:58,3:31,04/06/2024
...,...,...,...,...,...
497,Deisi,06:54,12:30,5:36,26/06/2024
498,Deisi,13:23,16:56,3:32,26/06/2024
499,Deisi,06:52,12:37,5:45,27/06/2024
500,Deisi,13:28,16:00,2:32,27/06/2024


In [27]:
totDATA = procesamiento(data)

display(totDATA)

Columnas renombradas exitosamente.


,Nombre,Minutos totales por dia,Fecha
0,Manuel F,301,01/06/2024
1,Manuel F,507,03/06/2024
2,Manuel F,554,04/06/2024
3,Manuel F,536,05/06/2024
4,Manuel F,490,06/06/2024
...,...,...,...
263,Deisi,336,19/06/2024
264,Deisi,526,20/06/2024
265,Deisi,548,26/06/2024
266,Deisi,497,27/06/2024


In [28]:
horas_extras(totDATA, mes="junio", feriados_csv="data/feriados_2024.csv")

10 feriados cargados desde data/feriados_2024.csv
Cálculo de horas extras completado.


In [29]:
display(totDATA)

,Nombre,Minutos totales por dia,Fecha,Sábado,Domingo/Feriado,Minutos extra 50%,Minutos extra 100%
0,Manuel F,301,01/06/2024,1,0,301,0
1,Manuel F,507,03/06/2024,0,0,27,0
2,Manuel F,554,04/06/2024,0,0,74,0
3,Manuel F,536,05/06/2024,0,0,56,0
4,Manuel F,490,06/06/2024,0,0,10,0
...,...,...,...,...,...,...,...
263,Deisi,336,19/06/2024,0,0,-144,0
264,Deisi,526,20/06/2024,0,0,46,0
265,Deisi,548,26/06/2024,0,0,68,0
266,Deisi,497,27/06/2024,0,0,17,0


In [30]:
out_name = "reporte_horas_extras_junio2024.csv"
generar_reporte(totDATA, file_name = out_name)

Directorio 'output_data' ha sido creado.
Reporte guardado exitosamente en output_data/reporte_horas_extras_junio2024.csv
